<a href="https://colab.research.google.com/github/e11106013/LLM/blob/main/Testing_royal_ZhTW_ID_q4_k_GGUFv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
#@title 安裝軟體套件
!pip install llama-cpp-python gradio

In [11]:
#@title 登入HF
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineG

In [12]:
#@title 下載自HF下載模型
!mkdir -p /content/models
#!huggingface-cli download roylin1003/royal-ZhTW-ID-q4_k_m --local-dir models
!huggingface-cli download roylin1003/royal-zhTW-ID-q2_k.gguf --local-dir models

Fetching 3 files:   0% 0/3 [00:00<?, ?it/s]Downloading 'README.md' to 'models/.cache/huggingface/download/Xn7B-BWUGOee2Y6hCZtEhtFu4BE=.05a9e268f05334bb93bbd1e07685a70a57228eea.incomplete'

.gitattributes: 100% 1.58k/1.58k [00:00<00:00, 5.94MB/s]

README.md: 100% 3.00k/3.00k [00:00<00:00, 22.6MB/s]
Download complete. Moving file to models/.gitattributes
Download complete. Moving file to models/README.md
Fetching 3 files:  33% 1/3 [00:00<00:00,  6.21it/s]
royal-zhTW-ID-q2_k.gguf:   0% 0.00/3.02G [00:00<?, ?B/s]
royal-zhTW-ID-q2_k.gguf:   0% 10.5M/3.02G [00:00<00:39, 75.4MB/s]
royal-zhTW-ID-q2_k.gguf:   1% 31.5M/3.02G [00:00<00:24, 123MB/s] 
royal-zhTW-ID-q2_k.gguf:   2% 52.4M/3.02G [00:00<00:26, 112MB/s]
royal-zhTW-ID-q2_k.gguf:   2% 73.4M/3.02G [00:00<00:26, 110MB/s]
royal-zhTW-ID-q2_k.gguf:   3% 94.4M/3.02G [00:00<00:31, 93.1MB/s]
royal-zhTW-ID-q2_k.gguf:   4% 115M/3.02G [00:02<01:35, 30.4MB/s] 
royal-zhTW-ID-q2_k.gguf:   4% 126M/3.02G [00:02<01:23, 34.7MB/s]
royal-zhTW-ID-q2_k.gguf:  

In [13]:
import gradio as gr
from llama_cpp import Llama

# --- 載入模型（使用 cache 避免重複載入） ---
def load_model():
    return Llama(
        model_path="models/royal-ZhTW-ID-q4_k_m.gguf",
      #  n_gpu_layers=20,
        n_gpu_layers=-1,    # 使用所有 GPU 層（-1 表示全部）

        n_ctx=2048
    )

llm = load_model()

# --- 翻譯函式 ---
def translate(input_text, direction, history):
    if direction == "華語 ➜ 印尼語":
        prompt = f"""請將下列華語句子精準翻譯為印尼語，只回傳印尼語內容，不要翻譯成其他語言：
華語：{input_text.strip()}
印尼語："""
        output_prefix = "印尼語："
    else:
        prompt = f"""請將下列印尼語句子精準翻譯為華語，只回傳華語內容，不要翻譯成其他語言：
印尼語：{input_text.strip()}
華語："""
        output_prefix = "華語："

    # 模型推論
    output = llm(prompt, max_tokens=100)
    result = output["choices"][0]["text"].strip()

    user_display = f"👤 {input_text.strip()}"
    bot_display = f"🤖 {output_prefix}{result}"

    history.append((user_display, bot_display))
    return "", history

# --- Gradio 介面設定 ---
with gr.Blocks(title="繁中印尼翻譯助理") as demo:
    gr.Markdown("# 🗣️ 繁中 ↔ 印尼 翻譯對話介面")
    gr.Markdown("使用模型：`roylin1003/royal-ZhTW-ID-q4_k_m`")

    direction = gr.Radio(
        ["華語 ➜ 印尼語", "印尼語 ➜ 華語"],
        label="請選擇翻譯方向",
        value="華語 ➜ 印尼語"
    )

    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="請輸入要翻譯的句子...", label="輸入")
    state = gr.State([])

    msg.submit(translate, [msg, direction, state], [msg, chatbot])

# --- 執行 ---
demo.launch()

llama_model_loader: loaded meta data with 27 key-value pairs and 339 tensors from models/royal-ZhTW-ID-q4_k_m.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Qwen2.5 7B Instruct
llama_model_loader: - kv   3:                       general.organization str              = Qwen
llama_model_loader: - kv   4:                           general.finetune str              = Instruct
llama_model_loader: - kv   5:                           general.basename str              = Qwen2.5
llama_model_loader: - kv   6:                         general.size_label str              = 7B
llama_model_loader: - kv   7:                          qwen

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5a896a995f26f833be.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
